In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 1. See Data

In [ ]:
train = pd.read_csv("../input/tabular-playground-series-may-2021/train.csv", index_col='id')
train.head()

In [ ]:
train_num_cols = train.shape[0]
train_num_idx = train.shape[1]
print(train.shape)
print(train_num_cols)
print(train_num_idx)

In [ ]:
train.describe()

In [ ]:
train.isnull().all()

There are no NA cells in "train.csv".

In [ ]:
test = pd.read_csv("../input/tabular-playground-series-may-2021/test.csv", index_col='id')
test.head()

In [ ]:
test_num_cols = test.shape[0]
test_num_idx = test.shape[1]
print(test.shape)
print(test_num_cols)
print(test_num_idx)

In [ ]:
test.describe()

In [ ]:
test.isnull().all()

There are no NA cells in "test.csv".

# 2. EDA (Explanatory Data Analysis) 

First of all, visualize class distribution in "train.csv".

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# create new DataFrme from Numpy
name = ["Class 1", "Class 2", "Class 3", "Class 4"]
df = pd.DataFrame(name, columns=["Name"])

# count every Class
cnt = []
class_1 = train[train["target"] == "Class_1"].shape[0]
cnt.append(class_1)
class_2 = train[train["target"] == "Class_2"].shape[0]
cnt.append(class_2)
class_3 = train[train["target"] == "Class_3"].shape[0]
cnt.append(class_3)
class_4 = train[train["target"] == "Class_4"].shape[0]
cnt.append(class_4)
df["Class"] = cnt

sns.barplot(x="Name", y="Class", data=df)
ax1 = plt.subplot(111)
ax1 = sns.barplot(x="Name", y="Class", data=df)
ax1.set(title="train target Distribution")

Perform data Standardization

In [ ]:
max_value = 0
min_value = 0
max_list = train.max()
min_list = train.min()

for i in range(0,50):
    if max_list[i] > max_value:
        max_value = max_list[i]
    if min_value > min_list[i]:
        min_value = min_list[i]
    
print(f"Max value in train data: {max_value}")
print(f"Min value in train data: {min_value}")

In [ ]:
def hist_plt(x):
    plt.figure()
    sns.displot(x, kde=False, bins=4)
    plt.xlim(-10,70)
    plt.ylim(0, 10000)
    plt.show()

for i in range(0,50):
    hist_plt(train.iloc[:,i])

In [ ]:
# Add new features
train_sum = train.sum(axis=1)
train_sum = train_sum.rename("sum")
train_mean = train.mean(axis=1)
train_mean = train_mean.rename("mean")
train_median = train.median(axis=1)
train_median = train_median.rename("median")
train_std = train.std(axis=1)
train_std = train_std.rename("std")

train_add = pd.concat([train, train_sum, train_mean, train_median, train_std], axis=1)
train_add.describe()

In [ ]:
# label encoding
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(train_add["target"])
train_add["target"] = le.transform(train["target"])
train_add.head()

In [ ]:
plt.figure(figsize=(16, 12))
sns.heatmap(train_add.corr(), cmap='Reds', square=True, linewidths=0.5) #.corr is pandas function which can be caluculated correlation, annot=True shows value, fmt is format
plt.show()

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
train_mod = train_add.drop("target", axis=1)

scaler.fit(train_mod)
# train_scaler and test_std are ndarray, need to transform to DataFrame
train_scaler = scaler.transform(train_mod)
train_scaler = pd.DataFrame(train_scaler)
target = train_add["target"].to_numpy()
train_scaler["target"] = target # add target column in train_scaler
print(train_scaler.describe())

# Add new features in test data
test_sum = test.sum(axis=1)
test_sum = test_sum.rename("sum")
test_mean = test.mean(axis=1)
test_mean = test_mean.rename("mean")
test_median = test.median(axis=1)
test_median = test_median.rename("median")
test_std = test.std(axis=1)
test_std = test_std.rename("std")

test_add = pd.concat([test, test_sum, test_mean, test_median, test_std], axis=1)
test_add.describe()
test_scaler = scaler.transform(test_add) # scaler should be based on train data, not test data
test_scaler = pd.DataFrame(test_scaler)
print()
print(test_scaler.describe()) # std is not equal to 1.0 but no problem

In [ ]:
pca = PCA(n_components=2)
pca.fit(train_scaler)
features = pca.transform(train_scaler)
pca_list = pca.explained_variance_ratio_
pca_df = pd.DataFrame(pca_list)
print(np.cumsum(pca_list)+[5])
print(list(np.cumsum(pca_list)+[5]))

In [ ]:
# PCA
from sklearn.decomposition import PCA

def PCA_plt(train_scaler, i):
    pca = PCA(n_components=i)
    pca.fit(train_scaler)
    features = pca.transform(train_scaler)
    pca_list = pca.explained_variance_ratio_
    pca_list = np.insert(pca_list, 0, 0)
    
    fig = plt.figure(figsize=(12, 8))

    title_ax1 = "PCA Analysis for train data " + "num " + str(i)
    ax1 = fig.add_subplot(121)
    ax1.scatter(features[:,0], features[:,1], alpha=0.5, c=list(train_add.loc[:,"target"]))
    ax1.grid()
    ax1.set_xlabel("PC1")
    ax1.set_ylabel("PC2")
    ax1.set(title=title_ax1)
    
    title_ax2 = "Contribution Rate " + "num " + str(i)
    ax2 = fig.add_subplot(122)
    ax2.plot(0, marker="o")
    ax2.plot(np.cumsum(pca_list), marker="o")
    ax2.grid()
    ax2.set_xlabel("Number of Components")
    ax2.set_ylabel("Cumulative Contribution")
    ax2.set(title=title_ax2)
    
    plt.show()
    
for i in range(2,54):
    PCA_plt(train_scaler, i)

In [ ]:
# Use CatBoost
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss
from catboost import CatBoostClassifier
from sklearn.svm import LinearSVC, SVC

#train_x = train_scaler.drop(["target"], axis=1)
train_x = train_scaler.iloc[:,51:54]
train_y = train_scaler["target"]

score_logloss = []

kf = KFold(n_splits=4, shuffle=True, random_state=0)
for tr_idx, va_idx in kf.split(train_x):
    tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
    tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]
    
    model = CatBoostClassifier(random_state=0)
    model.fit(tr_x, tr_y)
    
    va_pred = model.predict_proba(va_x)
    logloss = log_loss(va_y, va_pred)
    
    score_logloss.append(logloss)

logloss_mean = np.mean(score_logloss)
print(f'log loss score(mean): {logloss_mean:.5f}')

In [ ]:
test_data = test_scaler.iloc[:,51:54]
prediction = model.predict_proba(test_data)
submission = pd.DataFrame(prediction, columns=['Class_1','Class_2','Class_3','Class_4'])

sample_submission = pd.read_csv("../input/tabular-playground-series-may-2021/sample_submission.csv")
submission["id"] = sample_submission["id"]
submission = submission.reindex(columns=["id", "Class_1", "Class_2", "Class_3", "Class_4"])
submission.to_csv('submission.csv', index=False)
submission.head()